| 　 | 1 | 1, 1 | 1 | 2 |
| :--- | --- | --- | --- | --- |
| **2** | 　 | ■ | ■ | 　 |
| **1, 1** | ■ | 　 | 　 | ■ |
| **1, 1** | 　 | ■ | 　 | ■ |

In [1]:
def len_lines(qs):
    cont = False
    result = []
    for q in qs:
        if q == 0:
            cont = False
        else:
            if cont:
                result[-1] += 1
            else:
                result.append(1)
            cont = True
    return result

In [2]:
import dwavebinarycsp as dbc

row_constraints = [[2], [1, 1], [1, 1]]
col_constraints = [[1], [1, 1], [1], [2]]

csp = dbc.ConstraintSatisfactionProblem(dbc.BINARY)
for i, lengths in enumerate(row_constraints):
    csp.add_constraint(lambda *qs, lengths=lengths: len_lines(qs) == lengths, [f'q{i}{j}' for j in range(len(col_constraints))])
for j, lengths in enumerate(col_constraints):
    csp.add_constraint(lambda *qs, lengths=lengths: len_lines(qs) == lengths, [f'q{i}{j}' for i in range(len(row_constraints))])

bqm = dbc.stitch(csp)
bqm

BinaryQuadraticModel({'q00': 0.0, 'q01': -4.0, 'q02': -2.0, 'q03': -2.0, 'q10': -4.0, 'q11': 0.0, 'q12': -2.0, 'q13': -4.0, 'q20': -4.0, 'q21': -4.0, 'q22': -2.0, 'q23': -4.0}, {('q01', 'q00'): -4.0, ('q02', 'q00'): 4.0, ('q02', 'q01'): -2.0, ('q03', 'q00'): -2.0, ('q03', 'q01'): 4.0, ('q03', 'q02'): -2.0, ('q11', 'q10'): 4.0, ('q12', 'q10'): -2.0, ('q12', 'q11'): -2.0, ('q13', 'q10'): 0.0, ('q13', 'q11'): -2.0, ('q13', 'q12'): 4.0, ('q21', 'q20'): 4.0, ('q22', 'q20'): -2.0, ('q22', 'q21'): -2.0, ('q23', 'q20'): 0.0, ('q23', 'q21'): -2.0, ('q23', 'q22'): 4.0, ('q10', 'q00'): 4.0, ('q20', 'q00'): 4.0, ('q20', 'q10'): 4.0, ('q11', 'q01'): 4.0, ('q21', 'q01'): -4.0, ('q21', 'q11'): 4.0, ('q12', 'q02'): 4.0, ('q22', 'q02'): 4.0, ('q22', 'q12'): 4.0, ('q13', 'q03'): 0.0, ('q23', 'q03'): 4.0, ('q23', 'q13'): 0.0}, 7.5, Vartype.BINARY)

In [3]:
import sympy

expr = bqm.offset
for symb, coef in bqm.linear.items():
    expr += sympy.symbols(symb) * coef
for (symb1, symb2), coef in bqm.quadratic.items():
    expr += sympy.symbols(symb1) * sympy.symbols(symb2) * coef

sympy.init_printing()
expr

-4.0⋅q₀₀⋅q₀₁ + 4.0⋅q₀₀⋅q₀₂ - 2.0⋅q₀₀⋅q₀₃ + 4.0⋅q₀₀⋅q₁₀ + 4.0⋅q₀₀⋅q₂₀ - 2.0⋅q₀₁
⋅q₀₂ + 4.0⋅q₀₁⋅q₀₃ + 4.0⋅q₀₁⋅q₁₁ - 4.0⋅q₀₁⋅q₂₁ - 4.0⋅q₀₁ - 2.0⋅q₀₂⋅q₀₃ + 4.0⋅q
₀₂⋅q₁₂ + 4.0⋅q₀₂⋅q₂₂ - 2.0⋅q₀₂ + 4.0⋅q₀₃⋅q₂₃ - 2.0⋅q₀₃ + 4.0⋅q₁₀⋅q₁₁ - 2.0⋅q₁₀
⋅q₁₂ + 4.0⋅q₁₀⋅q₂₀ - 4.0⋅q₁₀ - 2.0⋅q₁₁⋅q₁₂ - 2.0⋅q₁₁⋅q₁₃ + 4.0⋅q₁₁⋅q₂₁ + 4.0⋅q
₁₂⋅q₁₃ + 4.0⋅q₁₂⋅q₂₂ - 2.0⋅q₁₂ - 4.0⋅q₁₃ + 4.0⋅q₂₀⋅q₂₁ - 2.0⋅q₂₀⋅q₂₂ - 4.0⋅q₂₀
 - 2.0⋅q₂₁⋅q₂₂ - 2.0⋅q₂₁⋅q₂₃ - 4.0⋅q₂₁ + 4.0⋅q₂₂⋅q₂₃ - 2.0⋅q₂₂ - 4.0⋅q₂₃ + 7.5

In [4]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(bqm, num_reads=100)

In [6]:
import pandas as pd

pd.DataFrame([{'check': csp.check(sample), 'energy': energy, 'occurr': num_occurrences, 'chain': chain_break_fraction, **sample} for sample, energy, num_occurrences, chain_break_fraction in response.data()])

,chain,check,energy,occurr,q00,q01,q02,q03,q10,q11,q12,q13,q20,q21,q22,q23
0,0.166667,True,-22.5,31,0,1,1,0,1,0,0,1,0,1,0,1
1,0.083333,True,-22.5,1,0,1,1,0,1,0,0,1,0,1,0,1
2,0.166667,False,-18.5,19,1,1,1,0,1,0,0,1,0,1,0,1
3,0.166667,False,-18.5,48,1,1,1,0,1,0,0,1,0,1,0,1
4,0.333333,False,-14.5,1,1,1,1,0,1,0,1,1,0,1,0,1


In [7]:
response.info['timing']

{'total_real_time': 24086,
 'qpu_access_overhead_time': 721,
 'anneal_time_per_run': 20,
 'post_processing_overhead_time': 2119,
 'qpu_sampling_time': 16396,
 'readout_time_per_run': 123,
 'qpu_delay_time_per_sample': 21,
 'qpu_anneal_time_per_sample': 20,
 'total_post_processing_time': 2119,
 'qpu_programming_time': 7690,
 'run_time_chip': 16396,
 'qpu_access_time': 24086,
 'qpu_readout_time_per_sample': 123}

In [8]:
sampler.properties['child_properties']['chip_id']

'DW_2000Q_2_1'

In [9]:
!pip list | grep dwave

dwave-cloud-client     0.4.16    
dwave-neal             0.4.2     
dwave-networkx         0.6.6     
dwave-ocean-sdk        1.0.3     
dwave-qbsolv           0.2.9     
dwave-system           0.5.4     
dwavebinarycsp         0.0.9     
